In [2]:
# Install dependencies
!pip install transformers tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import AdamW
from tqdm.notebook import tqdm
import numpy as np
from peft import LoraConfig, get_peft_model, TaskType

# Define a custom Dataset class
class MisconceptionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts.tolist() if isinstance(texts, pd.Series) else texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            max_length=self.max_len,
            add_special_tokens=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Prepare data and dataloaders
def prepare_data(file_path):
    # Load the dataset
    df = pd.read_csv(file_path)

    # Prepare the text data
    X = df[['answer', 'ConstructName', 'QuestionText']].astype(str)
    X = "answer: " + X['answer'] + " " + "ConstructName: " + X['ConstructName'] + " " + "QuestionText: " + X['QuestionText']

    # Use MisconceptionId as labels
    Y = df['MisconceptionId'].astype(int)

    # Encode the labels to start from 0
    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(Y)
    num_labels = len(label_encoder.classes_)

    # Split the data into training and temp (temp will be split into validation and test)
    X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)  # 80% train, 20% temp

    # Split temp into validation and test sets
    X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)  # 10% val, 10% test

    return X_train, Y_train, X_val, Y_val, X_test, Y_test, label_encoder, num_labels

def create_dataloaders(train_x, train_y, val_x, val_y, test_x, test_y, tokenizer, batch_size=16):
    # Prepare datasets
    train_dataset = MisconceptionDataset(train_x, train_y, tokenizer)
    val_dataset = MisconceptionDataset(val_x, val_y, tokenizer)
    test_dataset = MisconceptionDataset(test_x, test_y, tokenizer)

    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, val_dataloader, test_dataloader

# Train and evaluate
# Add the rest of the script here and make sure output files (e.g., model checkpoints) are saved to Google Drive

def train_and_evaluate(train_dataloader, val_dataloader, model, device, epochs=10, learning_rate=2e-5):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    best_loss = float('inf')
    model.to(device)

    num_training_steps = epochs * len(train_dataloader)
    with tqdm(total=num_training_steps, desc="Fine-tuning") as pbar:
        for epoch in range(epochs):
            # Training phase
            model.train()
            train_loss = 0

            for batch in train_dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                loss = loss_fn(logits, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                pbar.update(1)

            avg_train_loss = train_loss / len(train_dataloader)
            print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

            # Validation phase
            model.eval()
            val_loss = 0
            correct_predictions = 0
            total_predictions = 0

            with torch.no_grad():
                for batch in val_dataloader:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['label'].to(device)

                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                    logits = outputs.logits

                    loss = loss_fn(logits, labels)
                    val_loss += loss.item()

                    _, preds = torch.max(logits, dim=1)
                    correct_predictions += torch.sum(preds == labels)
                    total_predictions += labels.size(0)

            avg_val_loss = val_loss / len(val_dataloader)
            val_accuracy = correct_predictions.double() / total_predictions
            print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

            # Save the best model checkpoint
            if avg_val_loss < best_loss:
                best_loss = avg_val_loss
                best_model_state_dict = model.state_dict()
                torch.save(best_model_state_dict, 'best_model_checkpoint.pth')
                print(f"Best model saved with Validation Loss: {avg_val_loss:.4f}")

def evaluate_test_set(test_dataloader, model, device):
    model.eval()
    test_loss = 0
    correct_predictions = 0
    total_predictions = 0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = loss_fn(logits, labels)
            test_loss += loss.item()

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            total_predictions += labels.size(0)

    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_predictions.double() / total_predictions
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

def save_embeddings(dataloader, model, device, output_file="fine_tuned_embeddings.pt"):
    model.eval()
    embeddings_list = []
    labels_list = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Saving Embeddings"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].cpu().numpy()

            # Get the encoder outputs
            outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask)
            # Take the embeddings from the last hidden layer
            last_hidden_state = outputs.last_hidden_state
            # For CLS token, we take the first token
            cls_embeddings = last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
            embeddings_list.append(cls_embeddings.cpu())
            labels_list.extend(labels)

    embeddings = torch.cat(embeddings_list, dim=0)
    labels = np.array(labels_list)
    torch.save({'embeddings': embeddings, 'labels': labels}, output_file)
    print(f"Embeddings and labels saved to {output_file}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = '/content/drive/MyDrive/finalDataSet.csv'
X_train, Y_train, X_val, Y_val, X_test, Y_test, label_encoder, num_labels = prepare_data(file_path)
model_name = 'tbs17/MathBERT'  # Replace with the exact model name
tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)
lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,  # Sequence Classification task
        r=8,  # Low-rank dimension
        lora_alpha=32,  # Scaling factor
        lora_dropout=0.1,  # Dropout for LoRA layers
        bias="none"  # Options: "none", "all", "lora_only"
)

model = get_peft_model(base_model, lora_config)

train_dataloader, val_dataloader, test_dataloader = create_dataloaders(
    X_train, Y_train, X_val, Y_val, X_test, Y_test, tokenizer, batch_size=32
)

train_and_evaluate(train_dataloader, val_dataloader, model, device, epochs=10, learning_rate=2e-5)
model.load_state_dict(torch.load('best_model_checkpoint.pth'))
evaluate_test_set(test_dataloader, model, device)
save_embeddings(test_dataloader, model, device, output_file="fine_tuned_test_embeddings.pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 1/10, Train Loss: 7.3919
Epoch 1/10, Validation Loss: 7.3483, Validation Accuracy: 0.0023
Best model saved with Validation Loss: 7.3483
Epoch 2/10, Train Loss: 7.2079
Epoch 2/10, Validation Loss: 7.3392, Validation Accuracy: 0.0092
Best model saved with Validation Loss: 7.3392
Epoch 3/10, Train Loss: 7.0841
Epoch 3/10, Validation Loss: 7.3680, Validation Accuracy: 0.0183
Epoch 4/10, Train Loss: 6.9953
Epoch 4/10, Validation Loss: 7.4063, Validation Accuracy: 0.0206
Epoch 5/10, Train Loss: 6.9393
Epoch 5/10, Validation Loss: 7.4397, Validation Accuracy: 0.0229
Epoch 6/10, Train Loss: 6.9060
Epoch 6/10, Validation Loss: 7.4484, Validation Accuracy: 0.0092
Epoch 7/10, Train Loss: 6.8770
Epoch 7/10, Validation Loss: 7.4598, Validation Accuracy: 0.0114
Epoch 8/10, Train Loss: 6.8500
Epoch 8/10, Validation Loss: 7.4695, Validation Accuracy: 0.0137
Epoch 9/10, Train Loss: 6.8245
Epoch 9/10, Validation Loss: 7.4656, Validation Accuracy: 0.0137
Epoch 10/10, Train Loss: 6.8062
Epoch 10/10,

<ipython-input-3-82cb9d497a7f>:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_checkpoint.pth'))


Test Loss: 7.3764, Test Accuracy: 0.0160


Saving Embeddings:   0%|          | 0/14 [00:00<?, ?it/s]

Embeddings and labels saved to fine_tuned_test_embeddings.pt
